In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
'''
Takes too long to train, because there are too "many" features (26 of them are 1 feature). Need to clean up with PCA.
'''

In [2]:
featured_df = pd.read_csv("featured_df.csv", encoding="utf-8")
featured_df.head()

,x,y,num_vowels,num_words,num_digits,num_symbols,num_chars,is_plural,has_char_repeat,has_color,has_brand,has_descriptor,has_unit
0,LGHT NNFT YGRT,LIGHT NONFAT YOGURT,0,3,0,0,14,0,1,1,1,0,0
1,WRHDS CNDY WRMS,WARHEADS CANDY WORMS,0,3,0,0,15,1,0,0,1,1,0
2,WGMNS MN HRT,WEGMANS MINI HEART,0,3,0,0,12,0,0,0,1,0,0
3,RSN LMND CRNCH T CLSTRS,RAISIN ALMOND CRUNCH OAT CLUSTERS,0,5,0,0,23,1,0,0,1,0,0
4,WHT MT CRNBRRY PCN CHCKN SLD,WHITE MEAT CRANBERRY PECAN CHICKEN SALAD,0,6,0,0,28,0,1,0,1,0,0


In [3]:
limit = int(105722/5)
featured_df = featured_df.head(limit)
le = LabelEncoder()
y_transform = le.fit_transform(featured_df['y'].tolist())
y_transform.shape

(21144,)

In [4]:
## for now train with all x
## for now train with all y
all_cols = featured_df.columns.values.tolist()
all_cols.remove('x')
all_cols.remove('y')

train_x = featured_df[all_cols]
train_x.head()

,num_vowels,num_words,num_digits,num_symbols,num_chars,is_plural,has_char_repeat,has_color,has_brand,has_descriptor,has_unit
0,0,3,0,0,14,0,1,1,1,0,0
1,0,3,0,0,15,1,0,0,1,1,0
2,0,3,0,0,12,0,0,0,1,0,0
3,0,5,0,0,23,1,0,0,1,0,0
4,0,6,0,0,28,0,1,0,1,0,0


In [5]:
train_x.shape

(21144, 11)

In [6]:
featured_df['label'] = y_transform
train_y = featured_df['label']
featured_df.head()

,x,y,num_vowels,num_words,num_digits,num_symbols,num_chars,is_plural,has_char_repeat,has_color,has_brand,has_descriptor,has_unit,label
0,LGHT NNFT YGRT,LIGHT NONFAT YOGURT,0,3,0,0,14,0,1,1,1,0,0,10396
1,WRHDS CNDY WRMS,WARHEADS CANDY WORMS,0,3,0,0,15,1,0,0,1,1,0,20080
2,WGMNS MN HRT,WEGMANS MINI HEART,0,3,0,0,12,0,0,0,1,0,0,20134
3,RSN LMND CRNCH T CLSTRS,RAISIN ALMOND CRUNCH OAT CLUSTERS,0,5,0,0,23,1,0,0,1,0,0,15210
4,WHT MT CRNBRRY PCN CHCKN SLD,WHITE MEAT CRANBERRY PECAN CHICKEN SALAD,0,6,0,0,28,0,1,0,1,0,0,20413


In [7]:
train_x = np.array(train_x)
train_y = np.array(train_y)

In [8]:
gbdt = GradientBoostingClassifier(n_estimators=100, max_depth=3)

In [ ]:
%%time
'''
train on everything (currently 33% at once)
'''
gbdt.fit(train_x, train_y)

In [ ]:
shuffled = featured_df.sample(frac=1)
eval_df = shuffled.iloc[:1000]
preds = gbdt.predict(eval_df.drop(columns=['x', 'y', 'label'], axis=1))
preds

In [ ]:
eval_df['preds'] = le.inverse_transform(preds)

In [ ]:
eval_df[['x', 'y', 'preds']]